In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tracks = pd.read_csv('../augmented_ds.csv')
artists = pd.read_csv('../artists.csv')
artists['followers_log'] = np.log10(artists.followers + 1)

In [ ]:
%matplotlib inline

artists.followers.describe()

In [ ]:
artists.followers_log.describe()

In [ ]:
artists.followers.hist(log=True, bins=10)

In [ ]:
artists.followers_log.hist(bins=20)

Notice that 50% (!) of all artist on spotify have less than 393 followers, and 25% even have less than 45!

In [ ]:
artists.popularity.hist(bins=20, log=True)

Popularity is equally skewed as followers, but not by that much.

In [ ]:
pd.options.display.max_rows = 500
display(artists.sort_values(by='followers', ascending=False).head(40))

In [ ]:
display(artists.sort_values(by='popularity', ascending=False).head(40))

In [ ]:
p5, p95 = artists.followers_log.quantile([0.05, 0.95])
print(f'5% percentile: {p5}, 95% percentile: {p95}')

data_mat = artists.loc[(artists.followers_log > p5) & (artists.followers_log < p95)][['followers_log', 'popularity']].values
np.corrcoef(data_mat, rowvar=False)

In [ ]:
plt.scatter(data_mat[:,0] / np.std(data_mat[:,0]), data_mat[:, 1] / np.std(data_mat[:,1]), alpha= 0.2, s=0.6)

Popularity and followers are highly correlated! Maybe consider doing a PCA-like transformation to get a singular and corrected value. Notice as well, that there are some artist that are highly unpopular while still having a significant amount of followers. Are these dead people? Lets find out :)

In [ ]:
artists.loc[(artists.followers_log > 3.0) & (artists.popularity < 0.05)].sort_values('followers')

Include probably famous people from platform like youtube or insta, but who are shitty musicians. For instance, minecraft troll song (Lachlan - Mobs cant handle me).

To filter these "indie artists", simply thresholding 'followers' or 'popularity' is not sufficient. Instead we need a PCA to remove the outsider of the distribution.

In [ ]:
X = (data_mat - np.mean(data_mat, axis=0, keepdims=True)) / np.std(data_mat, axis=0, keepdims=True)

In [ ]:
V, Sigma, _ = np.linalg.svd(np.cov(X, rowvar=False), hermitian=True)

In [ ]:
tmp = X @ V
plt.scatter(tmp[:, 0], tmp[:, 1], s=0.1)

In [ ]:
tmp2 = tmp[np.fabs(tmp[:,1]) < 0.8]  @ V.T
plt.scatter(tmp2[:, 0], tmp2[:, 1], s=0.1) 

In [ ]:
print(np.sum(np.fabs(tmp[:,1]) > 0.8))

In [ ]:
plt.scatter(X[:, 0], X[:,1], s=0.1)

In [ ]:
np.cov(X, rowvar=False)

In [ ]:
Vt

In [ ]:
np.diag(Sigma